# Subtitle 

Création de la plage des sous-titres au format SRT pour les videos.

[Informations modèle Whisper (Open-AI)](https://github.com/openai/whisper)

In [ ]:
path = "/Users/gauthier/Desktop/video_test.mp4"

In [ ]:
try:
    import whisper
    print("✅ Whisper est installé — version :", whisper.__version__)
except ImportError:
    print("❌ Whisper n'est pas installé.")
    !pip install -U openai-whisper

In [ ]:
import math
import textwrap
import whisper

In [ ]:
# Le modèle turbo est un bon equilibre qualité/vitesse, voir large pour les cas spécifiques.
# whisper.available_models()

In [ ]:
model = whisper.load_model("turbo")
result = model.transcribe(path)

### Ecriture de fichier de transcription brut pour la lecture.

In [ ]:


txt_path = path + ".txt"

with open(txt_path, "w", encoding="utf-8") as f:
    for segment in result['segments']:
        parole = segment["text"]
        f.write(parole + "\n")

In [ ]:
def _format_ts_srt(t: float) -> str:
    """Formate un timestamp en HH:MM:SS,mmm (SRT)."""
    ms = int(round((t % 1) * 1000))
    secs = int(math.floor(t))
    if ms == 1000:  # évite 99,1000 -> 100,000
        secs += 1
        ms = 0
    h = secs // 3600
    m = (secs % 3600) // 60
    s = secs % 60
    return f"{h:02}:{m:02}:{s:02},{ms:03}"

def to_srt(segments, max_line_length: int | None = None) -> str:
    """Convertit une liste de segments Whisper en sous-titres SRT."""
    out, idx = [], 1
    for seg in segments:
        text = (seg.get("text") or "").strip()
        if not text:
            continue
        if max_line_length:
            text = "\n".join(textwrap.wrap(text, width=max_line_length))
        out.append(str(idx))
        out.append(f"{_format_ts_srt(seg['start'])} --> {_format_ts_srt(seg['end'])}")
        out.append(text)
        out.append("")  # ligne vide entre blocs
        idx += 1
    return "\n".join(out)

### Écriture du fichier SRT (Sous-titre)

In [ ]:
srt_text = to_srt(result.get("segments"))

srt_path = path + ".srt"

with open(srt_path, "w", encoding="utf-8") as f:
    f.write(srt_text)